<a href="https://colab.research.google.com/github/HanhaodiZhang/HATE/blob/main/mlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq transformers

In [ ]:
import torch
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("olid-training.tsv", sep='\t', header=0, index_col='id')

df.head()

,tweet,subtask_a,subtask_b,subtask_c
id,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [ ]:
df_test_label = pd.read_csv("labels-levela.csv",header=None,names=['id','subtask_a'],index_col='id')
df_test_tweet = pd.read_csv("testset-levela.tsv", sep='\t', header=0, index_col='id')
df_test = pd.merge(df_test_tweet, df_test_label, on=['id'])
df_test.head()

,tweet,subtask_a
id,,
15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF
27014,"#ConstitutionDay is revered by Conservatives, ...",NOT
30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT
13876,#Watching #Boomer getting the news that she is...,NOT
60133,#NoPasaran: Unity demo to oppose the far-right...,OFF


In [ ]:
df['comment']= df['tweet']
df_test['comment'] = df_test['tweet']

In [ ]:

# M = df['comment']
 
# M.to_csv('train.txt',header=False,index=False)
# M = df_test['comment']
# M.to_csv('val.txt', header=False, index=False)

with open("train.txt", 'w') as f:
    for line in df['comment'].to_list():
        f.write(line + '\n')

with open("val.txt", 'w') as f:
    for line in df_test['comment'].to_list():
        f.write(line + '\n')

In [ ]:
from transformers import BertTokenizer

PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
# tokenizer.add_special_tokens({'additional_special_tokens':["<OFF>","<NOT>"]})

In [ ]:
from transformers import BertForMaskedLM

model = BertForMaskedLM.from_pretrained('bert-base-uncased')
# model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# TRAIN ONLY
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling

train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="train.txt",
    block_size=128,
)

valid_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="val.txt",
    block_size=128,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:128: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
# TRAIN ONLY
from transformers import Trainer, TrainingArguments, EvalPrediction

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)
    
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

model.cuda()
training_args = TrainingArguments(
    output_dir="./mlm3",
    overwrite_output_dir=True,
    num_train_epochs=15,
    per_device_train_batch_size=16,
    save_steps=1000,
    save_total_limit=10,
    evaluation_strategy='epoch',
    learning_rate=1e-4,
    weight_decay=1e-5,
    lr_scheduler_type='cosine',
    warmup_steps=500,
    metric_for_best_model='eval_loss',
    load_best_model_at_end=True,
    disable_tqdm=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    # compute_metrics=compute_metrics
    # prediction_loss_only=True,
)

In [ ]:
%%time
# TRAIN ONLY
trainer.train()

{'loss': 2.3263, 'learning_rate': 0.0001, 'epoch': 0.6}
{'eval_loss': 1.8276678323745728, 'eval_runtime': 1.7615, 'eval_samples_per_second': 488.217, 'epoch': 1.0}
{'loss': 1.1274, 'learning_rate': 9.956649043661442e-05, 'epoch': 1.21}
{'loss': 1.0868, 'learning_rate': 9.827347896811953e-05, 'epoch': 1.81}
{'eval_loss': 1.8446189165115356, 'eval_runtime': 1.7965, 'eval_samples_per_second': 478.715, 'epoch': 2.0}
{'loss': 1.0815, 'learning_rate': 9.614338690800175e-05, 'epoch': 2.42}
{'eval_loss': 1.824037790298462, 'eval_runtime': 1.9383, 'eval_samples_per_second': 443.679, 'epoch': 3.0}
{'loss': 1.0419, 'learning_rate': 9.321315086741916e-05, 'epoch': 3.02}
{'loss': 0.995, 'learning_rate': 8.953358226023457e-05, 'epoch': 3.62}
{'eval_loss': 1.7677382230758667, 'eval_runtime': 1.7833, 'eval_samples_per_second': 482.239, 'epoch': 4.0}
{'loss': 0.9903, 'learning_rate': 8.516848621366188e-05, 'epoch': 4.23}
{'loss': 0.9623, 'learning_rate': 8.019355516295254e-05, 'epoch': 4.83}
{'eval_los

TrainOutput(global_step=12420, training_loss=0.9036793314124459, metrics={'train_runtime': 1801.9626, 'train_samples_per_second': 6.892, 'epoch': 15.0})

In [ ]:
import torch.nn.functional as F
import torch
from transformers import BertTokenizer
from transformers import BertForMaskedLM
from tqdm.notebook import tqdm

PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
model = BertForMaskedLM.from_pretrained(PRE_TRAINED_MODEL_NAME)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.load_state_dict(torch.load("./mlm3/pytorch_model.bin")) # model path
model.to('cuda')
model.eval()

tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
tokenizer.add_special_tokens({'additional_special_tokens':["@USER"]}) # add @USER to avoid cutting into '@ USER'

model.resize_token_embeddings(len(tokenizer))

Embedding(30523, 768)

In [ ]:
# top=5 限制最大概率的5个词，生成时随机从其中sample
def get_prediction(model, masked_input, tokenizer, top=5):
    tokenized_inputs = tokenizer(masked_input, return_tensors='pt', add_special_tokens=True, return_attention_mask=True)

    mask_id = tokenizer.mask_token_id

    input_ids = tokenized_inputs['input_ids'].to('cuda')
    token_type_ids = tokenized_inputs['token_type_ids'].to('cuda')
    attention_mask = tokenized_inputs['attention_mask'].to('cuda')

    mask_positions = input_ids == mask_id

    result = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)

    logits = result.logits
    
    mask_logits = F.softmax(logits[mask_positions], dim=-1)
    mask_preds = torch.argsort(mask_logits, dim=-1, descending=True)
    mask_preds = mask_preds[:, 0:top].tolist()
    
    predition_tokens = []
    for preds in mask_preds:
        tokens = tokenizer.convert_ids_to_tokens(preds)
        tokens = [i for i in tokens if i != '[UNK]'] # remove UNK
        predition_tokens.append(tokens)
    return predition_tokens

get_prediction(model, "You are [MASK]", tokenizer)

[['gorgeous', 'beautiful', 'amazing', 'adorable']]

In [ ]:
import random
random.seed(17)
import copy

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from string import punctuation
STOPWORDS = set(stopwords.words('english')) | set(punctuation) | {'@USER', '[UNK]'}


# randomly mask noun
def mask_without_stopwords(text, n=1):
    mask_candidate_positions = []
    stop_word_positions = []
    words = []
    
    for i, word in enumerate(tokenizer.tokenize(text)):
        if word not in STOPWORDS:
            mask_candidate_positions.append(i)
        else:
            stop_word_positions.append(i)
        words.append(word)
    
    if len(mask_candidate_positions) < n:
        mask_candidate_positions += random.sample(stop_word_positions, n - len(mask_candidate_positions))

    mask_positions = random.sample(mask_candidate_positions, n)
    mask_words=[]
    for mask_pos in mask_positions:
        mask_words.append(words[mask_pos])
        words[mask_pos] = '[MASK]'
        
        
    return tokenizer.convert_tokens_to_string(words),mask_words

def sample_MLM_prediction(text,maskwords,predictions, K=1):
    result = []
    mask_preds = []

    for prediction in predictions[0]:
        if prediction != maskwords[0]:
          mask_preds.append(prediction)
    
    # for i in range(K):
    #     temp = text.split()
    #     tweet=''
    #     for word in temp:
    #         if word== maskwords:
    #           word= mask_preds[mask_num]
    #         tweet = " ".join(word)   
    #     result.append(tweet)
    try:
      for i in range(K):
        temp = text.split()
        tweet=''
        tweet_list=[]
        for word in temp:
          if word== maskwords[0]:

            word= mask_preds[0][i]
          tweet_list.append(word)
        tweet = " ".join(tweet_list)   
        result.append(tweet)
    except IndexError as error:
      pass

    return result

text='@USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER Shush it makes the ship make more sense'
masked_text,maskwords = mask_without_stopwords('@USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER Shush it makes the ship make more sense', n=1)
predictions = get_prediction(model, masked_text, tokenizer)
aug = sample_MLM_prediction(text,maskwords, predictions, 2)
print(masked_text)
print("----------------------------------------")
print(predictions)
print("----------------------------------------")
print(aug)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
@USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER shush it makes the ship [MASK] more sense
----------------------------------------
[['make', 'making', 'have', 'no', 'even']]
----------------------------------------
['@USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER Shush it makes the ship m more sense', '@USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER Shush it makes the ship a more sense']


In [ ]:
masked_text,maskwords = mask_without_stopwords('@USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER Shush it makes the ship make more sense', n=1)
predictions = get_prediction(model, masked_text, tokenizer)

In [ ]:
N=1
K=2

In [ ]:
df_aug = pd.DataFrame({
    'id':[],
    'tweet':[],
    'subtask_a':[],
})
df_aug = df_aug.set_index('id')

In [ ]:
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    label = row['subtask_a'] # use same label
    masked_tweet,maskwords  = mask_without_stopwords(row['tweet'], N)

    mask_predictions = get_prediction(model, masked_tweet, tokenizer)

    # print([masked_tweet])
    # print(mask_predictions)
    augmented_texts = sample_MLM_prediction(row['tweet'],maskwords, mask_predictions, K)
    
    for i, text in enumerate(augmented_texts):
        id = int(index*10 + i + 1)
        df_aug = df_aug.append([{'id':id,'tweet':text.replace(" ##", ''),'subtask_a':label}],ignore_index=False)


In [ ]:
df_aug = df_aug.set_index('id')
df_aug.to_csv("bert_aug_no_tag_N={}_k={}.csv".format(N,K))

In [ ]:
result = []
mask_preds = []
for sorted_prediction in predictions:
        preds = random.sample(sorted_prediction, 2)
        mask_preds.append(preds)
mask_preds

[['made', 'make']]

In [ ]:
maskwords

['make']

In [ ]:
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    label = row['subtask_a'] # use same label
    masked_tweet = mask_without_stopwords(row['tweet'], N)

    mask_predictions, mask_words = get_prediction(model, masked_tweet, tokenizer)

    # print([masked_tweet])
    # print(mask_predictions)
    augmented_texts = sample_MLM_prediction(masked_tweet, mask_predictions, K)
    
    for i, text in enumerate(augmented_texts):
        id = int(index*10 + i + 1)
        df_aug = df_aug.append([{'id':id,'tweet':text.replace(" ##", ''),'subtask_a':label}],ignore_index=False)